In [138]:
import Datasets as DS
import warnings
import numpy as np
import ce_helpers
from itertools import chain
warnings.filterwarnings("ignore")
from importlib import reload

In [139]:
wd = '/Users/tabearober/Documents/Counterfactuals/CE-OCL/data/'

In [140]:
reload(DS)

<module 'Datasets' from '/Users/tabearober/Documents/Counterfactuals/CE-OCL/Datasets.py'>

In [141]:
df, df_train, df_test, d, clf, df_factuals, model_master, encoder, scaler = DS.adult(wd)

In [142]:
target = d['target']
numerical = d['numerical']
categorical = d['categorical']

mapping = {}
for i in range(len(categorical)):
    mapping[categorical[i]] = [categorical[i] + '_' + s for s in list(encoder.categories_[i])]
mapping

{'marital-status': ['marital-status_Married', 'marital-status_Non-Married'],
 'native-country': ['native-country_Non-US', 'native-country_US'],
 'occupation': ['occupation_Managerial-Specialist', 'occupation_Other'],
 'race': ['race_Non-White', 'race_White'],
 'relationship': ['relationship_Husband', 'relationship_Non-Husband'],
 'sex': ['sex_Female', 'sex_Male'],
 'workclass': ['workclass_Non-Private', 'workclass_Private']}

## recreate original dataset

This is not necessary at this point.

In [143]:
# df_dummies, df_orig = DS.recreate_orig(df, d, encoder)

## Optimization

### Optimization Prep

X: complete dataset without target

In [144]:
X = df.drop(target,axis=1)

X1: points in X that have 1 as label. They will be used as trust region

In [145]:
y_ix_1 = np.where(df[target]==1)
X1 = X.iloc[y_ix_1[0],:].copy().reset_index(drop=True, inplace=False)

Factual instance

In [146]:
u_index = 0
u = df_factuals.drop(target,axis=1).iloc[u_index,:]
# print(u)
print('predicted label: %d' % (clf.predict([u])))

predicted label: 0


Specify parameters

In [147]:
sp = True
mu = 10000
tr_region = True
enlarge_tr = False
num_counterfactuals = 3

In [148]:
# features that can only increase (become larger)
L = []
# conditionally mutable features, such as education level (can only take on higher categories)
Pers_I = []
# features that can only be positive
P = []
# immutable features
# I = mapping[d['immutable']]
I = [mapping.get(key) for key in list(set(d['immutable']).intersection(categorical))]
I = list(set(list(chain.from_iterable(I))).intersection(df.columns))
# dictionary with one-hot encoding used to ensure coherence
F_coh = {}
# integer features
F_int = []
# categorical features one-hot encoded
F_b = df.columns.difference(numerical + [target])
# F_r
F_r = numerical

### Optimize for all factuals

In [149]:
df_performance = pd.DataFrame()
eval_final = pd.DataFrame(columns=['validity', 'cat_prox', 'cont_prox', 'sparsity', 'cat_diver',
       'cont_diver', 'cont_count_divers'])

for u_index in range(len(df_factuals)):
# for u_index in range(4):
    print('u_index: %i' % u_index)
    u = df_factuals.drop(target,axis=1).iloc[u_index,:]

    CEs, CEs_, final_model = ce_helpers.opt(X, X1, u, numerical, F_b, F_int, F_coh, I, L, Pers_I, P, sp, mu,
                           tr_region, enlarge_tr, num_counterfactuals, model_master, scaler)

    CEs_[target] = clf.predict(CEs.drop('scaled_distance', axis=1))
    df_dummies, df_orig = DS.recreate_orig(CEs_, d, encoder)

    df_performance_1 = ce_helpers.evaluation(clf, df_orig.drop(target,axis=1), numerical, categorical, rounding=True, CEs_=CEs)

    df_performance = df_performance.append(df_performance_1)

eval_final.loc['mean'] = np.array(df_performance.mean())
eval_final.loc['std'] = np.array(df_performance.std())

eval_final

u_index: 0
Generating constraints for the trust region using 11687 samples.
The trust region is not being enlarged.
... Trust region defined.
Embedding constraints for counterfactual_adult
Changed value of parameter PoolSolutions to 103
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 20000.19466132187
The optimal solution is: [0.315068493, 0.230798686, 0.7666676805638417, 0.08296182913181838, -1.8189894035458565e-12, 0.34693877599999956, 1.0, 1.0, -0.0, 1.0, 1.0, 0.0, 1.0]
u_index: 1
Generating constraints for the trust region using 11687 samples.
The trust region is not being enlarged.
... Trust region defined.
Embedding constraints for counterfactual_adult
Changed value of parameter PoolSolutions to 103
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 10000.005097480389
The optimal solution is: [0.3287

,validity,cat_prox,cont_prox,sparsity,cat_diver,cont_diver,cont_count_divers
mean,0.989000,0.993667,-16083.432333,0.865000,0.005000,8635.841500,0.132500
std,0.060249,0.026844,28605.389024,0.060615,0.022361,11448.861246,0.061462


In [ ]:
eval_final.round(2)

---

# for just one factual

In [87]:
CEs, CEs_, final_model = ce_helpers.opt(X, X1, u, numerical, F_b, F_int, F_coh, I, L, Pers_I, P, sp, mu,
                           tr_region, enlarge_tr, num_counterfactuals, model_master, scaler)

Generating constraints for the trust region using 107686 samples.
The trust region is not being enlarged.
... Trust region defined.
Embedding constraints for counterfactual_give_me_some_credit
Changed value of parameter PoolSolutions to 103
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 10000.022499699331
The optimal solution is: [0.001129723000000027, 0.9178082190000001, 0.19999999999999996, 0.0011747739999999895, 0.05210050300000002, 0.416666667, 0.15000100223369373, 0.0, 0.125, 0.0]


## Inspect and Visualise

We create df_dummies and df_orig.

The model trained by CARLA uses one-hot encoding and drops the reference categories from the df. Hence, we have only sex_Male in the dataframe, but not sex_Female. df_dummies includes all dummy variables, and df_orig is a dataframe that is mapped back into the original input space. So in df_orig, we have a variable 'sex' with values 'Male' and 'Female', instead of dummy variables.

We can use df_orig to visualise our counterfactuals.

In [88]:
CEs_[target] = clf.predict(CEs.drop('scaled_distance', axis=1))
df_dummies, df_orig = DS.recreate_orig(CEs_, d, encoder)

In [89]:
ce_helpers.visualise_changes(df_orig)

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
original,0.8481,87.9994,2.0,1.5588,2591.975,10.0008,3.0,0.0,1.0,0.0,0
sol0,-,-,-,-,-,-,1.5,-,-,-,-
sol1,-,-,-,-,-,-,0.5,-,-,-,1.0
sol2,0.6939,-,-,-,-,-,2.5,-,-,-,-


## Evaluation

In [90]:
df_performance = ce_helpers.evaluation(clf, df_orig.drop(target,axis=1), numerical, categorical, rounding=True, CEs_=CEs)
df_performance

,validity,cat_prox,cont_prox,sparsity,cat_diver,cont_diver,cont_count_divers
0,0.33,None,-1.55,0.87,None,1.44,0.17
